Loader

In [4]:
from vector_mapping import EmbeddingLoader

EMBEDDING_PATH_ROOT = "./pretrained_word2vec/"
EN_EMBEDDING = "en/GoogleNews-vectors-negative300.bin.gz"
ZH_EMBEDDING = "zh/sgns.merge.word.bz2"

MAX_VOCAB = 10000

loader = EmbeddingLoader()

en_emb = loader.load_word2vec(filepath=EMBEDDING_PATH_ROOT+EN_EMBEDDING, max_vocab=MAX_VOCAB)
zh_emb = loader.load_word2vec(filepath=EMBEDDING_PATH_ROOT+ZH_EMBEDDING, max_vocab=MAX_VOCAB)

In [ ]:
import numpy as np

def get_word_embedding(word, embeddings):
    if word not in embeddings:
        raise KeyError(f"Word '{word}' not found in embeddings")
    return embeddings[word]


def cosine_similarity(vec1, vec2):
    # Calculate dot product
    dot_product = np.dot(vec1, vec2)

    # Calculate magnitudes (L2 norms)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    # Avoid division by zero
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0

    # Calculate cosine similarity
    return dot_product / (norm_vec1 * norm_vec2)

In [7]:
similarity = cosine_similarity(get_word_embedding("一", zh_emb), get_word_embedding("one", en_emb))

print(similarity)

0.11028192


In [9]:
similarity = cosine_similarity(get_word_embedding("一", zh_emb), get_word_embedding("一个", zh_emb))

print(similarity)

0.684622


In [14]:
similarity = cosine_similarity(get_word_embedding("one", en_emb), get_word_embedding("single", en_emb))

print(similarity)

0.4903487
